In [1]:
import pandas as pd
import numpy as np
from faker import Faker
fake = Faker()

In [20]:
#import transition_matrix
transition_matrix = pd.read_csv("transition_matrix.csv")
transition_matrix.set_index("location", inplace=True)
transition_matrix

,checkout,dairy,drinks,fruit,spices,entrance
location,,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000,0.0
dairy,0.088776,0.746961,0.061634,0.050891,0.051739,0.0
drinks,0.208109,0.011236,0.610650,0.090865,0.079140,0.0
entrance,0.000000,0.284727,0.160332,0.359364,0.195577,0.0
fruit,0.203077,0.086538,0.050000,0.613462,0.046923,0.0
spices,0.143429,0.188301,0.173878,0.095353,0.399038,0.0


In [3]:
class Customer:
    ''' a single customer that moves through the supermarket in a MCMC simulation'''
    
    def __init__(self, c_id, name, state, budget=np.random.randint(100,500, dtype=int)):
        self.c_id = c_id
        self.name = name
        self.state = state
        self.budget = budget
        self.transition_matrix = transition_matrix
        self.hist = [self.state]
        self.money = [self.budget]
    
    def __repr__(self):
        if self.hist[-1] == 'entrance' or self.hist[-1] == 'checkout':
            return  f'Customer {self.name} in {self.state} with {self.budget} $'
        else:
            return f'Customer {self.c_id}, {self.name} in {self.state} with {self.budget} $ left, was in {self.hist[-2]}'
    
    def is_active(self):
        """
        Returns True if the customer has not reached the checkout
        for the second time yet, False otherwise.
        """
        if self.state == 'checkout':
            return f'Customer {self.name} has been churned and spent {cust1.money[0] - cust1.money[-1]}$!'
        else:
            return f'Customer {self.name} is still active.'
        
    
    def next_state(self):
        '''
        Propagates the customer to the next state.
        '''

        next_loc = np.random.choice(self.transition_matrix.columns.values, p=self.transition_matrix.loc[self.state])
        #p = Probs
        
        #hard coded the revenue
        if next_loc == 'fruit':
            self.budget -= 4
        elif next_loc == 'spices':
            self.budget -= 3
        elif next_loc == 'dairy':
            self.budget -= 5
        elif next_loc == 'drinks':
            self.budget -= 6
        
        #list for 'path-history'
        self.state = next_loc
        self.hist.append(self.state)
        
        #list for money spend
        self.budget = self.budget
        self.money.append(self.budget)

In [4]:
class Supermarket:
    def __init__(self): 
        self.customer = []
        self.c_id = 0
        self.hour = 7
        self.minutes = 0
        #self.current_time = self.get_time() #doesn't work
        self.data = []
        
        
    
    def __repr__(self):
        return f'There are {len(self.customer)} in the supermarket. Time: {self.get_time()}.'
    
    def add_customer(self): #works
        self.c_id += 1
        self.customer.append(Customer(self.c_id, fake.name(), 'entrance', np.random.randint(1,500)))
    
    def get_time(self):
        """current time in HH:MM format,
        """
        #the supermarket opens at 7
        hour = self.hour
        minutes = self.minutes
        return f'{hour}:{minutes}'
        #append time
        
    def next_minute(self):
        """propagates all customers to the next state.
        """
        self.minutes += 1
        self.move_customers()
        self.add_customer()
        self.write_all_cust()
        self.remove_existing_customers()
        
        
        
    def move_customers(self):
        for cust in self.customer:
            cust.next_state()
    
    def remove_existing_customers(self):
        """removes every customer that is not active any more.
        """
        for cust in self.customer:
            if cust.state == 'checkout':
                self.customer.remove(cust)
                print(f'Customer {cust.name} has been churned!')
    
    def write_all_cust(self):
        for cust in self.customer:
            self.data.append([cust.c_id, cust.name, cust.state, cust.budget, self.get_time()])
        
        

In [5]:
sm = Supermarket()

In [6]:
sm

There are 0 in the supermarket. Time: 7:0.

In [7]:
sm.get_time()

'7:0'

In [14]:
sm.customer

[Customer 3, Alicia Doyle in dairy with 315 $ left, was in dairy,
 Customer 4, Teresa Norton in dairy with 373 $ left, was in dairy,
 Customer 5, Nancy Keller in fruit with 325 $ left, was in entrance,
 Customer Thomas Harrington in entrance with 395 $]

In [13]:
sm.next_minute() #move the cust

Customer Cody Gonzales has been churned!


In [15]:
sm.data

[[1, 'Austin Oliver', 'entrance', 332, '7:1'],
 [1, 'Austin Oliver', 'spices', 329, '7:2'],
 [2, 'Cody Gonzales', 'entrance', 429, '7:2'],
 [1, 'Austin Oliver', 'spices', 326, '7:3'],
 [2, 'Cody Gonzales', 'dairy', 424, '7:3'],
 [3, 'Alicia Doyle', 'entrance', 329, '7:3'],
 [1, 'Austin Oliver', 'checkout', 326, '7:4'],
 [2, 'Cody Gonzales', 'dairy', 419, '7:4'],
 [3, 'Alicia Doyle', 'fruit', 325, '7:4'],
 [4, 'Teresa Norton', 'entrance', 383, '7:4'],
 [2, 'Cody Gonzales', 'dairy', 414, '7:5'],
 [3, 'Alicia Doyle', 'dairy', 320, '7:5'],
 [4, 'Teresa Norton', 'dairy', 378, '7:5'],
 [5, 'Nancy Keller', 'entrance', 329, '7:5'],
 [2, 'Cody Gonzales', 'checkout', 414, '7:6'],
 [3, 'Alicia Doyle', 'dairy', 315, '7:6'],
 [4, 'Teresa Norton', 'dairy', 373, '7:6'],
 [5, 'Nancy Keller', 'fruit', 325, '7:6'],
 [6, 'Thomas Harrington', 'entrance', 395, '7:6']]

In [16]:
our_sm = pd.DataFrame.from_records(sm.data, columns=['cust_id', 'cust_name', 'cust_state', 'budget', 'timestamp'])
our_sm['timestamp'] = pd.to_datetime(our_sm['timestamp'], format= '%H:%M').dt.time

In [19]:
our_sm.head()

,cust_id,cust_name,cust_state,budget,timestamp
0,1,Austin Oliver,entrance,332,07:01:00
1,1,Austin Oliver,spices,329,07:02:00
2,2,Cody Gonzales,entrance,429,07:02:00
3,1,Austin Oliver,spices,326,07:03:00
4,2,Cody Gonzales,dairy,424,07:03:00


In [18]:
our_sm[our_sm['cust_id'] == 2]

,cust_id,cust_name,cust_state,budget,timestamp
2,2,Cody Gonzales,entrance,429,07:02:00
4,2,Cody Gonzales,dairy,424,07:03:00
7,2,Cody Gonzales,dairy,419,07:04:00
10,2,Cody Gonzales,dairy,414,07:05:00
14,2,Cody Gonzales,checkout,414,07:06:00


In [ ]:
Supermarket1=Supermarket(7)

In [ ]:
Supermarket1

In [ ]:
Supermarket1.add_new_customers

In [ ]:
print(Supermarket.write_all_cust)

In [ ]:
print(Supermarket)

In [ ]:
for i in customers: #move all customers
    i.next_state()
    if i.state == 'checkout':
        print(f'Customer {i.name} has been churned!') #adding time?
        customers.remove(i)
    else:
        print(i)
    if len(customers) < 20: #new_customers
            x = 20 - len(customers)
            #new_cust = []
            new_cust = ([Customer(fake.name(), 'entrance', np.random.randint(1,500)) for i in range(x)])
            for c in new_cust:
                customers.append(c)

In [ ]:
print(Supermarket)

In [ ]:
print_customers()

In [ ]:
    def add_new_customers(self, stop, id_suffix, terrain_map, image, x, y):
        """randomly creates new customers.
        """
        for i in range(stop):
            cust = Customer(str(i) + "_" + str(id_suffix), "entrance", transition_matrix,
                            terrain_map=terrain_map, image=image, x=x, y=y)
            self.customers.append(cust)

        self.id_suffix += 1

In [ ]:
from faker import Faker

In [ ]:
import datetime

In [ ]:
datetitime(7, 26)

In [ ]:
datetime.datetime.now().time()

In [ ]:
import time

In [ ]:
time.time(7, 26)

In [ ]:
time.strftime("%a, %d %b %Y %H:%M:%S")

In [ ]:
time.strftime("%H:%M:%S")